<a href="https://colab.research.google.com/github/blurred421/LFD473-code/blob/main/labs/jb_Lab%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Instructions

In the lab, you're presented a task such as building a dataset, training a model, or writing a training loop, and we'll provide the code structured in such a way that you can fill in the blanks in the code using the knowledge you acquired in the chapters that precede the lab. You should be able to find appropriate snippets of code in the course content that work well in the lab with minor or no adjustments.

The blanks in the code are indicated by ellipsis (`...`) and comments (`# write your code here`).

In some cases, we'll provide you partial code to ensure the right variables are populated and any code that follows it runs accordingly.

```python
# write your code here
x = ...
```

The solution should be a single statement that replaces the ellipsis, such as:

```python
# write your code here
x = [0, 1, 2]
```

In some other cases, when there is no new variable being created, the blanks are shown like in the example below:

```python
# write your code here
...
```

Although we're showing you only a single ellipsis (`...`), you may have to write more than one line of code to complete the step, such as:

```python
# write your code here
for i, xi in enumerate(x):
    x[i] = xi * 2
```

## 7.8 Lab 3: Classifying Images

Now it is YOUR turn to classify some images! First, you will need to choose and load a [model for image classification](https://pytorch.org/vision/stable/models.html#classification) and its corresponding [weights](https://pytorch.org/vision/stable/models.html#table-of-all-available-classification-weights).

Don't forget to retrieve the prescribed transformation function or model corresponding to the model you chose. Also, take a look at its size and accuracy, so you have an idea of its performance.

TIP: try a very small model (e.g. MobileNet) and a very large model (e.g. VGG) and see how long they take to run inference on your images.

### 7.8.1 Load Weights

Load the weights from the model of your choice into its own object:

In [46]:
#  May want to check out this website/api for images:  https://unsplash.com/developers


from torchvision.models import get_weight

# write your code here
weights = get_weight('MobileNet_V3_Small_Weights.DEFAULT')

### 7.8.2 Load Model

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step1.png)

Load the model using Torch Hub and the weights you've just loaded:

In [ ]:
import torch
repo = 'pytorch/vision'

# write your code here
model = torch.hub.load(repo, 'mobilenet_v3_small', weights=weights)

Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 66.3MB/s]


### 7.8.3 Extract Metadata

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

Retrieve the categories used to pretrain the model, and the transformation function that should be applied to the input images:

In [ ]:
# write your code here
categories = weights.meta['categories']
transforms_fn = weights.transforms()

transforms_fn

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

Let's inspect the number of parameters and the metrics of the model you chose. Run the two cells below as they are to visualize their output:

In [ ]:
weights.meta['num_params']/1e6

2.542856

In [ ]:
weights.meta['_metrics']

{'ImageNet-1K': {'acc@1': 67.668, 'acc@5': 87.402}}

### 7.8.4 Making Predictions

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

Now, let's use the pretrained model you've already loaded to make predictions for an image. First, though, let's download an image from Wikipedia once again. The downloading function is reproduced below for convenience.

In [ ]:
import numpy as np
import requests
from PIL import Image
from io import BytesIO

def get_image_from_url(url, headers=None):
    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))
    return img

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/7/72/Igel.JPG'
img = get_image_from_url(url)
img.height, img.width

(846, 1075)

Remember, the model cannot take PIL images as inputs, you need to preprocess the image first:

In [ ]:
# write your code here
preprocessed_img = transforms_fn(img)

Moreover, models expect mini-batches, not single images as inputs. Make sure you have a mini-batch with the right shape (N, C, H, W), standing for number of images in the mini-batch (one, in our case), number of channels in the image, its height, and its width:

In [ ]:
# write your code here
mini_batch = preprocessed_img.unsqueeze(0)
mini_batch.shape

torch.Size([1, 3, 224, 224])

You can use the mini-batch and the pretrained model to make predictions now:

In [ ]:
# The mini-batch above has a single data point
# Call the model and get the corresponding predictions(logits)
# write your code here
logit = model(mini_batch)[0]

# Fetch the index of the highest logit
# write your code here
idx = logit.argmax()

# Find the corresponding category
categories[idx]

'groom'

Perhaps you've figured it out that we forgot to set the model to evaluation mode. That shoudl fix it:

In [ ]:
# Set the model to evaluation mode
# write your code here
model.eval()


# Then find the predicted category as above
# write your code here
logit = model(mini_batch)[0]
idx = logit.argmax()
categories[idx]

'porcupine'

#### 7.8.4.1 Probabilities

In many cases, it may be interesting to return the probabilities next to the predictions. Convert the logits produced by the model into probabilities:

In [ ]:
import torch.nn.functional as F

# write your code here
probabilities = F.softmax(logit, dim=0)
probabilities

tensor([1.0329e-13, 3.6955e-17, 2.4925e-17, 1.0021e-16, 1.1528e-15, 2.8486e-13,
        9.0883e-13, 5.9741e-12, 3.3040e-10, 6.8928e-12, 1.3547e-13, 6.2444e-12,
        5.6574e-13, 1.0639e-12, 4.6894e-13, 3.8030e-16, 9.9499e-13, 1.0967e-13,
        1.4335e-12, 4.4977e-14, 5.1692e-13, 2.0577e-12, 1.1808e-13, 4.8461e-12,
        3.3553e-13, 7.8965e-16, 7.4297e-17, 1.4954e-15, 3.7244e-15, 1.5733e-16,
        5.7226e-15, 1.3489e-15, 5.6601e-15, 3.1830e-17, 3.4425e-15, 8.8839e-14,
        6.1755e-15, 3.2175e-14, 5.3052e-17, 6.9987e-15, 7.1730e-15, 8.7250e-15,
        1.2340e-14, 8.5258e-15, 1.9840e-13, 2.6265e-13, 2.4225e-14, 6.6432e-16,
        1.1115e-12, 1.3638e-15, 6.8803e-15, 8.4276e-14, 1.0537e-14, 2.6745e-16,
        1.7853e-12, 9.1716e-15, 1.7690e-14, 7.6355e-14, 1.1701e-14, 7.8326e-14,
        9.7861e-13, 2.5894e-14, 2.0973e-14, 3.6883e-12, 4.5869e-15, 1.0450e-15,
        1.7195e-15, 4.8575e-12, 3.5163e-15, 6.0358e-15, 1.1010e-14, 5.2586e-16,
        2.3044e-15, 5.2318e-14, 2.1084e-

Use PyTorch's own built-in function to get the top-K probabilities and corresponding indices:

In [ ]:
# write your code here
values, indices = torch.topk(probabilities, 1)
values, indices

(tensor([0.9999], grad_fn=<TopkBackward0>), tensor([334]))

The target or label is the class corresponding to the index above. Just run the cell below as is to visualize its output:

In [ ]:
categories[indices[0]]

'porcupine'

#### 7.8.4.2 Testing

In a real-world deployment, you won't have the input data neatly assembled as a dataset. You will have to create a mini-batch of the user's input data, feed it to the model to get its predicted logits, and then convert them into one or more predictions and probabilities that need to be returned to the user.

Write a function that takes either an URL or a filepath, a model, its prescribed transformations, and a list of target categories, and returns a list of the top K predictions:

In [ ]:
def predict(path_or_url, model, transforms_fn, categories, topk=1, headers=None):
    if path_or_url.startswith('http'):
        img = get_image_from_url(path_or_url, headers=headers)
    else:
        img = Image.open(path_or_url)

    # Apply the transformation to the image
    # write your code here
    preproc_img = transforms_fn(img)

    # If the transformation doesn't return a mini-batch
    # We make one ourselves by unsqueezing the first dimension
    if len(preproc_img.shape) == 3:
        preproc_img = preproc_img.unsqueeze(0)

    # Set the model to evaluation mode
    # write your code here
    model.eval()

    device = next(model.parameters()).device

    # Make predictions (logits)
    pred = model(preproc_img)

    # Compute probabilities out of the predicted logits
    # and then get the topk values and indices
    # write your code here
    probabilities = torch.nn.functional.softmax(pred[0], dim=0)
    values, indices = torch.topk(probabilities, topk)

    return [{'label': categories[i], 'value': v.item()} for i, v in zip(indices, values)]

Use the metadata from your model's weights as arguments to the function you wrote, and let's make a prediction using an image's URL:

In [ ]:
# write your code here
transforms_fn = weights.transforms()
categories = weights.meta['categories']

Let's make a prediction using an image's URL:

In [ ]:
from IPython.display import HTML

url = 'https://upload.wikimedia.org/wikipedia/commons/c/ce/Daisy_G%C3%A4nsebl%C3%BCmchen_Bellis_perennis_01.jpg'

HTML(f'<img src="{url}" width="300px">')

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/c/ce/Daisy_G%C3%A4nsebl%C3%BCmchen_Bellis_perennis_01.jpg'
# Complying with Wikimedia User Agent's policy: https://meta.wikimedia.org/wiki/User-Agent_policy
headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}

# Call the predict function on an URL of an image, like the one above
# Don't forget to pass the headers as argument
# write your code here
predict(url, model, transforms_fn, categories, headers=headers)

[{'label': 'daisy', 'value': 0.9999676942825317}]